# Preprocessing

### Preperation
Import libraries and create the pandas dataframes

In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

weather_df = pd.read_csv('data/weather_features.csv')
weather_df['dt_iso'] = pd.to_datetime(weather_df['dt_iso'])
weather_df = weather_df.applymap(lambda x: x.strip() if type(x)==str else x)

valencia_df = weather_df[weather_df['city_name'] == 'Valencia']
madrid_df = weather_df[weather_df['city_name'] == 'Madrid']
bilbao_df = weather_df[weather_df['city_name'] == 'Bilbao']
barcelona_df = weather_df[weather_df['city_name'] == 'Barcelona']
seville_df = weather_df[weather_df['city_name'] == 'Seville']

dataframes = [valencia_df, madrid_df, bilbao_df, barcelona_df, seville_df]
all_column_names = {}

for df in dataframes:
    city_name = df['city_name'].iloc[0]
    all_column_names[city_name] = []
    for column in valencia_df.columns:
        if valencia_df[column].dtype == np.float64:
            all_column_names[city_name].append(column)


In [56]:
valencia_df = valencia_df.loc[:, (valencia_df.fillna(0) != 0).any(axis=0)]
madrid_df = madrid_df.loc[:, (madrid_df.fillna(0) != 0).any(axis=0)]
bilbao_df = bilbao_df.loc[:, (bilbao_df.fillna(0) != 0).any(axis=0)]
barcelona_df = barcelona_df.loc[:, (barcelona_df.fillna(0) != 0).any(axis=0)]
seville_df = seville_df.loc[:, (seville_df.fillna(0) != 0).any(axis=0)]

In [57]:
valencia_df = valencia_df.interpolate(method='linear')
madrid_df = madrid_df.interpolate(method='linear')
bilbao_df = bilbao_df.interpolate(method='linear')
barcelona_df = barcelona_df.interpolate(method='linear')
seville_df = seville_df.interpolate(method='linear')

In [58]:
window_percentage = 3

# Function to get the upper and lower bands
def get_bands(data):
        upper_band = np.mean(data) + (3 * np.std(data))
        lower_band = np.mean(data) - (3 * np.std(data))
        return upper_band, lower_band

for df in dataframes:
    column_names = all_column_names[df['city_name'][0]]
    anomaly_masks = {}
    for column_name in column_names[1:]:
        column = df[column_name]
        N = len(column)
        k = int(N * (window_percentage/2/100))

        # Calculate the bands for each data point
        bands = [get_bands(column[max(0, i-k):min(i+k, N)]) for i in range(N)]
        upper, lower = zip(*bands)

        # Identify the anomalies
        anomalies = (column > upper) | (column < lower)
        anomaly_masks[column_name] = anomalies

    # Combine anomaly masks using bitwise OR (|) for each column's mask
    combined_mask = np.column_stack(list(anomaly_masks.values())).any(axis=1)

    # Filter the data
    filtered_data = df[~combined_mask]


{'Valencia': ['temp', 'temp_min', 'temp_max', 'rain_1h', 'rain_3h', 'snow_3h'], 'Madrid': ['temp', 'temp_min', 'temp_max', 'rain_1h', 'rain_3h', 'snow_3h'], 'Bilbao': ['temp', 'temp_min', 'temp_max', 'rain_1h', 'rain_3h', 'snow_3h'], 'Barcelona': ['temp', 'temp_min', 'temp_max', 'rain_1h', 'rain_3h', 'snow_3h'], 'Seville': ['temp', 'temp_min', 'temp_max', 'rain_1h', 'rain_3h', 'snow_3h']}
{'Valencia': ['temp', 'temp_min', 'temp_max', 'rain_1h', 'rain_3h', 'snow_3h'], 'Madrid': ['temp', 'temp_min', 'temp_max', 'rain_1h', 'rain_3h', 'snow_3h'], 'Bilbao': ['temp', 'temp_min', 'temp_max', 'rain_1h', 'rain_3h', 'snow_3h'], 'Barcelona': ['temp', 'temp_min', 'temp_max', 'rain_1h', 'rain_3h', 'snow_3h'], 'Seville': ['temp', 'temp_min', 'temp_max', 'rain_1h', 'rain_3h', 'snow_3h']}


KeyError: 0